In [159]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import average_precision_score

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.10f}'.format

FEATURE_DIR_NAME = '../data/features/'
RAW_DATA_DIR_NAME = '../data/raw/'

In [162]:
sorted(os.listdir('./oof'))

['.DS_Store',
 'cat_StratifiedKFold_06061151depth5_pl_train.pkl',
 'cat_StratifiedKFold_06061151depth5_pl_train_minmax.pkl',
 'cat_StratifiedKFold_06061151depth5_pl_train_normalized.pkl',
 'cat_StratifiedKFold_06061405depth5_pl_train.pkl',
 'cat_StratifiedKFold_06061405depth5_pl_train_minmax.pkl',
 'cat_StratifiedKFold_06061405depth5_pl_train_normalized.pkl',
 'lgb_StratifiedKFold_06061146_train.pkl',
 'lgb_StratifiedKFold_06061146_train_minmax.pkl',
 'lgb_StratifiedKFold_06061146_train_normalized.pkl',
 'lgb_StratifiedKFold_06061325cat_lgbm_pl45_55_add_train.pkl',
 'lgb_StratifiedKFold_06061325cat_lgbm_pl45_55_add_train_minmax.pkl',
 'lgb_StratifiedKFold_06061325cat_lgbm_pl45_55_add_train_normalized.pkl']

In [134]:
camaro_48 = pd.DataFrame(np.load('./048_wave_regularized3_noise010_with_layout_features_pseudo.csv.npy'))
camaro_66 = pd.DataFrame(np.load('./066_convnet1d_noise020_l64_k7_d8_with_layout_pseudo.csv.npy'))

In [163]:
oof_lgb_train = pd.read_pickle('./oof/lgb_StratifiedKFold_06061146_train.pkl')
oof_lgb_train_minmax = pd.read_pickle('./oof/lgb_StratifiedKFold_06061146_train_minmax.pkl')
oof_lgb_train_normalized = pd.read_pickle('./oof/lgb_StratifiedKFold_06061146_train_normalized.pkl')

# oof_cat_train = pd.read_pickle('./oof/cat_StratifiedKFold_06061151depth5_pl_train.pkl')
# oof_cat_train_minmax = pd.read_pickle('./oof/cat_StratifiedKFold_06061151depth5_pl_train_minmax.pkl')
# oof_cat_train_normalized = pd.read_pickle('./oof/cat_StratifiedKFold_06061151depth5_pl_train_normalized.pkl')

oof_cat_train = pd.read_pickle('./oof/cat_StratifiedKFold_06061405depth5_pl_train.pkl')
oof_cat_train_minmax = pd.read_pickle('./oof/cat_StratifiedKFold_06061405depth5_pl_train_minmax.pkl')
oof_cat_train_normalized = pd.read_pickle('./oof/cat_StratifiedKFold_06061405depth5_pl_train_normalized.pkl')

oof_lgb_pl_catlgb_train = pd.read_pickle('./oof/lgb_StratifiedKFold_06061325cat_lgbm_pl45_55_add_train.pkl')
oof_lgb_pl_catlgb_train_minmax = pd.read_pickle('./oof/lgb_StratifiedKFold_06061325cat_lgbm_pl45_55_add_train_minmax.pkl')
oof_lgb_pl_catlgb_train_normalized = pd.read_pickle('./oof/lgb_StratifiedKFold_06061325cat_lgbm_pl45_55_add_train_normalized.pkl')

In [164]:
# 正解ラベルのロード
train_y = pd.read_pickle(FEATURE_DIR_NAME + 'y_train.pkl')

In [165]:
train_y['target'].value_counts()

0    7200
1     236
Name: target, dtype: int64

## PLありのLightGBM

In [167]:
print(
    f'default:\t{average_precision_score(train_y, oof_lgb_train)}',
    f'\nmin_max:\t{average_precision_score(train_y, oof_lgb_train_minmax)}',
    f'\nStandardScaler:\t{average_precision_score(train_y, oof_lgb_train_normalized)}'
)

default:	0.9051061160048421 
min_max:	0.904967585678312 
StandardScaler:	0.9051613415697916


## PLありのCatboost

In [168]:
print(
    f'default:\t{average_precision_score(train_y, oof_cat_train)}',
    f'\nmin_max:\t{average_precision_score(train_y, oof_cat_train_minmax)}',
    f'\nStandardScaler:\t{average_precision_score(train_y, oof_cat_train_normalized)}'
)

default:	0.8871612665239644 
min_max:	0.8887905618104579 
StandardScaler:	0.8902497235225347


## 上記2つをブレンド

In [173]:
# ブレンド
oof_blend = oof_lgb_train*0.9 + oof_cat_train*0.1
oof_blend_minmax = oof_lgb_train_minmax*0.55 + oof_cat_train_minmax*0.45
oof_blend_ss = oof_lgb_train_normalized*0.55 + oof_cat_train_normalized*0.45

In [174]:
print(
    f'default:\t{average_precision_score(train_y, oof_blend)}',
    f'\nmin_max:\t{average_precision_score(train_y, oof_blend_minmax)}',
    f'\nStandardScaler:\t{average_precision_score(train_y, oof_blend_ss)}'
)

default:	0.9049052791334014 
min_max:	0.9037320752910738 
StandardScaler:	0.9049944919593425


## 上記ブレンドデータをPLデータとして追加したLGBM

In [157]:
print(
    f'default:\t{average_precision_score(train_y, oof_lgb_pl_catlgb_train)}',
    f'\nmin_max:\t{average_precision_score(train_y, oof_lgb_pl_catlgb_train_minmax)}',
    f'\nStandardScaler:\t{average_precision_score(train_y, oof_lgb_pl_catlgb_train_normalized)}'
)

default:	0.9069293428460025 
min_max:	0.9059888562757371 
StandardScaler:	0.9075442254923912


## camaroさんのoof

In [158]:
print(
    f'{average_precision_score(train_y, camaro_48)}',
    f'\n{average_precision_score(train_y, camaro_66)}'
)

0.8940894204355896 
0.8936653815051839


In [72]:
display(oof_train.head(), oof_train.shape)

,0
0,0.0000171599
1,0.0046559672
2,0.0000889775
3,0.0000067947
4,0.0004675498


(7436, 1)

In [73]:
display(oof_train_minmax.head(), oof_train_minmax.shape)

,0
0,-0.0292912292
1,-0.0239382916
2,-0.0285083757
3,-0.0301629981
4,-0.0297020539


(7436, 1)

In [74]:
display(oof_train_normalized.head(), oof_train_normalized.shape)

,0
0,-0.1959036281
1,-0.1644406145
2,-0.1958341429
3,-0.1967369047
4,-0.1937304147


(7436, 1)

In [75]:
oof_train.describe()

,0
count,7436.0000000000
mean,0.0286172907
std,0.1484364213
min,0.0000000111
25%,0.0000318658
50%,0.0001977353
75%,0.0011133245
max,0.9996744061


In [76]:
oof_train_minmax.describe()

,0
count,7436.0000000000
mean,0.0000000000
std,0.1485130290
min,-0.0301697585
25%,-0.0291454888
50%,-0.0284382857
75%,-0.0262232821
max,0.9737119118


In [77]:
oof_train_normalized.describe()

,0
count,7436.0000000000
mean,0.0000000000
std,1.0000672472
min,-0.1967809986
25%,-0.1956347970
50%,-0.1920875646
75%,-0.1802093929
max,6.7037630318
